In [3]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

np.random.seed(1)

def sigmoid(Z):
    return 1/(1+np.exp(-Z)), Z

def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA*s*(1-s)
    return dZ

def relu(Z):
    A = np.maximum(0, Z)
    return A, Z

def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z<=0] = 0
    return dZ

def init_params(n_x,n_h,n_y):
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros(n_h, 1)
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros(n_y,1)
    params = {"W1":W1,"W2":W2,"b1":b1, "b2":b2}
    return params

def init_params_deep(layer_dims):
    np.random.seed(3)
    params = {}
    L = len(layer_dims)
    for l in range(1, L):
        params["W"+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        params["b"+str(l)]=np.zeros(layer_dims[l],1)
    return params

def linear_forward(A, W, b):
    Z = np.dot(W, A) +b
    cache = (A, W, b)
    return Z, cache

def linear_activation_forward(A_prev,W,b,activation):
    Z,linear_cache = linear_forward(A_prew, W, b)
    if activation=='sigmoid':
        A, activation_cache = sigmoid(Z)
    elif activation=='relu':
        A, activation_cache = relu(Z)
    cache = (linear_cache, activation_cache)
    return A, cache

def L_model_forward(X, params):
    caches = []
    A = X
    L = len(params)//2
    for l in range(1, L):
        A_prev = A
        A,cache = linear_activation_forward(A_prev, params["W"+str(l)],
                                            params["b"+str(l)],'relu')
        caches.append(cache)
        
    AL,cache = linear_activation_forward(A_prev, params["W"+str(l)],
                                        params["b"+str(l)],'sigmoid')
    caches.append(cache)
    return AL, caches

def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -np.sum(np.multiply(np.log(AL),Y)+np.multiply(1-Y,np.log(1-AL)))/m
    cost = np.squeeze(cost)
    return cost

def linear_backward(dZ, cache):
    a_prev, W, b = cache
    m = a_prev.shape[1]
    dW = np.dot(dZ, a_prev.T)/m
    db = np.sum(dZ, axis=1, keepdims=True)/m
    da_prev = np.dot(W.T, dZ)
    return da_prev,dW,db

def linear_activation_backward(dA,cache, activation):
    linear_cache,activation_cache = cache
    if activation=='sigmoid':
        dZ = sigmoid_backward(dA, activation_cache)
    elif activation=='relu':
        dZ = relu_backward(dA, activation_cache)
    da_prev, dW, db = linear_backward(dZ, linear_cache)
    return da_prev,dW, db


    
if __name__ == '__main__':
    a,b = relu(2)
    print(a,b)

2 2
